In [130]:
import numpy as np
import re
from pprint import pp

sample = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

def get_array(data):
    width = len(data.splitlines()[0])
    digits = re.findall(r'\d', data)
    array = np.array(digits, dtype=int)
    array.shape = len(digits) // width, width
    return array


In [141]:
def get_neighbors(shape):
    results = {}
    for y in range(shape[0]):
        for x in range(shape[1]):
            neighbors = []
            if y > 0:
                neighbors.append((y - 1,x))
            if y < shape[0] - 1:
                neighbors.append((y + 1,x))
            if x > 0:
                neighbors.append((y,x - 1))
            if x < shape[1] - 1:
                neighbors.append((y,x + 1))
            results[(y,x)] = tuple(neighbors)
    return results

class Distancer:
    def __init__(self):
        self.max_y = self.array.shape[0] - 1
        self.max_x = self.array.shape[1] - 1
    def get_dist(self, y, x):
        return self.max_y - y + self.max_x - x

class DistancerFromData(Distancer):
    def __init__(self, data):
        self.array = get_array(data)
        super().__init__()

class DistancerFromArray(Distancer):
    def __init__(self, array):
        self.array = array
        super().__init__()

In [132]:
def visualize_risk(array, knowns):
    print("What")
    plot = np.zeros(array.shape, dtype=int)
    plot = plot - 1
    for known, risk in knowns.items():
        plot[known] = risk
    for row in plot:
        print (''.join(f'{str(i):>3}' if i > -1 else '  X' for i in row))
    print (" --- ")

In [156]:

from operator import itemgetter
from queue import PriorityQueue

third = itemgetter(2)

class NewPathFinder:
    def __init__(self, data, debug=False, limit=None):
        self.limit = limit
        self.debug = debug
        self.array = get_array(data)
        self.neighbors = get_neighbors(self.array.shape)
        self.max_y = self.array.shape[0] - 1
        self.max_x = self.array.shape[1] - 1
        self.end = self.max_y, self.max_x
        self.knowns = {self.end: 0}
        self.todo = PriorityQueue()
        self.todo.put((0, 0, self.end))
        self.count = 0
    
    def get_dist(self, y, x):
        return self.max_y - y + self.max_x - x

    def get_todo(self):
        while True:
            if self.limit and self.count > self.limit:
                print ("Limit reached")
                break
            self.count+=1
            if not self.todo.empty():
                yield third(self.todo.get())
            else:
                break

    def do_stuff(self):
        for todo in self.get_todo():
            risk_to_me = self.array[todo]
            my_risk = self.knowns[todo]
            self.debug and print(f'Regarding {todo}.  Known risk is {my_risk}')
            neighbors = self.neighbors[todo]
            for neighbor in neighbors:
                known = self.knowns.get(neighbor)
                new_risk = risk_to_me + my_risk
                if not known is not None:
                    self.debug and print (f"NEW! {neighbor}.  New risk is {new_risk}")
                    self.knowns[neighbor] = new_risk
                    self.todo.put((self.get_dist(*neighbor), self.get_dist(*neighbor), neighbor))
                elif new_risk < known:
                    self.debug and print (f"IMPROVED! {neighbor} {known} with {new_risk}")
                    self.knowns[neighbor] = new_risk
                else: 
                    self.debug and print (f"ALREADY GOOD! {neighbor} {known} is better than {new_risk}")


pf = NewPathFinder(sample, limit=200, debug=False)
print(pf.array)
pf.do_stuff()

visualize_risk(pf.array, pf.knowns)
print(pf.knowns[0,0])
print(pf.count)

[[1 1 6 3 7 5 1 7 4 2]
 [1 3 8 1 3 7 3 6 7 2]
 [2 1 3 6 5 1 1 3 2 8]
 [3 6 9 4 9 3 1 5 6 9]
 [7 4 6 3 4 1 7 1 1 1]
 [1 3 1 9 1 2 8 1 3 7]
 [1 3 5 9 9 1 2 4 2 1]
 [3 1 2 5 4 2 1 6 3 9]
 [1 2 9 3 1 3 8 5 2 1]
 [2 3 1 1 9 4 4 5 8 1]]
What
 40 40 34 31 30 25 24 25 28 32
 39 37 31 30 27 22 21 22 21 28
 37 36 33 27 22 21 20 19 19 21
 39 36 30 26 22 19 19 14 13 19
 33 32 26 23 19 18 14 13 12 13
 32 29 28 19 18 16 13 12  9 12
 31 28 27 25 16 15 13  9  7  9
 28 27 25 20 16 14 13  7  4  2
 31 28 24 21 20 16  9  4  2  1
 36 33 32 24 21 18 14  9  1  0
 --- 
40
101


In [147]:

pf = NewPathFinder(sample, limit=8, debug=True)
print(pf.array)
from pprint import pprint
pprint(pf.knowns)
pf.do_stuff()

visualize_risk(pf.array, pf.knowns)
print(pf.count)

[[1 1 6 3 7 5 1 7 4 2]
 [1 3 8 1 3 7 3 6 7 2]
 [2 1 3 6 5 1 1 3 2 8]
 [3 6 9 4 9 3 1 5 6 9]
 [7 4 6 3 4 1 7 1 1 1]
 [1 3 1 9 1 2 8 1 3 7]
 [1 3 5 9 9 1 2 4 2 1]
 [3 1 2 5 4 2 1 6 3 9]
 [1 2 9 3 1 3 8 5 2 1]
 [2 3 1 1 9 4 4 5 8 1]]
{(9, 9): 0}
Regarding (9, 9).  Known risk is 0
NEW! (8, 9).  New risk is 1
NEW! (9, 8).  New risk is 1
Regarding (8, 9).  Known risk is 1
NEW! (7, 9).  New risk is 2
ALREADY GOOD! (9, 9) 0 is better than 2
NEW! (8, 8).  New risk is 2
Regarding (9, 8).  Known risk is 1
ALREADY GOOD! (8, 8) 2 is better than 9
NEW! (9, 7).  New risk is 9
ALREADY GOOD! (9, 9) 0 is better than 9
Regarding (7, 9).  Known risk is 2
NEW! (6, 9).  New risk is 11
ALREADY GOOD! (8, 9) 1 is better than 11
NEW! (7, 8).  New risk is 11
Regarding (8, 8).  Known risk is 2
IMPROVED! (7, 8) 11 with 4
ALREADY GOOD! (9, 8) 1 is better than 4
NEW! (8, 7).  New risk is 4
ALREADY GOOD! (8, 9) 1 is better than 4
Regarding (9, 7).  Known risk is 9
ALREADY GOOD! (8, 7) 4 is better than 14
NEW! (9, 6).

In [158]:
realdeal = open('d15.input').read()
pf = NewPathFinder(realdeal, limit=99999, debug=False)
pf.do_stuff()
print(pf.count)
print(pf.knowns.get((0,0)))

10001
790
